<a href="https://colab.research.google.com/github/Sylvariane/La-faim-dans-le-monde/blob/master/P03_01_coder%C3%A9ponses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Réalisez une étude de santé publique

## Code des questions 1 à 14

In [ ]:
# On importe les deux librairies nécessaires à la réalisation de ce projet
import numpy as np
import pandas as pd

In [ ]:
# Ensuite, on importe les csv nécessaires à l'analyse
population_fao = pd.read_csv('fr_population.csv')
animaux_fao = pd.read_csv('fr_animaux.csv')
cereales_fao = pd.read_csv('fr_cereales.csv')
sousalim_fao = pd.read_csv('fr_sousalimentation.csv')
vegetaux_fao = pd.read_csv('fr_vegetaux.csv')

*Créez un dataframe contenant les informations de population de chaque pays. Calculez le nombre total d’humains sur la planète. Critiquez votre résultat. En cas d’anomalie, analysez et effectuer les corrections nécessaires.*

In [ ]:
# On ne garde que les colonnes qui nous seront utiles pour le reste de l'analyse
population_2013 = population_fao[['Zone', 'Code zone', 'Année','Valeur']]
population_2013 = population_2013.rename(columns={'Zone' : 'pays', 
                                                  'Code zone' : 'code_pays',
                                                  'Année' : 'annee',
                                                  'Valeur' : 'population'})
# On supprime les données en double
population_2013 = population_2013.drop_duplicates()
print("Population totale en 2013 : " + str(population_2013['population'].sum(skipna=True)* 1000))

Population totale en 2013 : 8413993000


En 2013, selon les données de la FAO, on dénombre 8.413.993.000 habitants sur Terre. Cette valeur correspond au total de la population mondiale calculé à l'aide du DataFrame construit à partir des données présentes dans les documents téléchargeables. Cependant sur d'autres sites le population mondiale est de 7.210.582.000.
De plus, en observant les données présentes sur le site de la FAO, on observe que la Chine est comptabilisée deux fois : une fois comme république démocratique de Chine et une autre fois en division (Chine continentale, Taïwan, Hong Kong et Macao). Il s'agit donc d'un doublon. Pour régler ce problème, on peut donc supprimer la Chine pour  calculer la population mondiale. Dans la suite de l'analyse, la Chine sera aussi enlevée. 

**Question 1** : donnez le résultat de votre calcul pour l'année 2013.

In [ ]:
# On crée un masque pour enlever les donneés redondantes
masque = population_2013['pays'] == 'Chine'

# On applique le masque pour avoir une nouvelle estimation de la population
population = population_2013[~masque]
print("Population totale en 2013 (après correction) : " + str(population['population'].sum(skipna=True)* 1000))

Population totale en 2013 (après correction) : 6997326000


Après le retrait de la population de la Chine, on obtient une population mondiale de 6.997.326.000. On va donc conserver ce chiffre pour le reste de l'analyse et ainsi supprimer les redondances qui pourraient apparaître. Cette valeur correspond à celle que l'on retrouve sur le site de la FAO lorsque l'on consulte l'ancienne méthodologie. 

---------------------------

Pour plus de simplicité, on va créer un DataFrame qui contiendra toutes les informations des bilans alimentaires animaux et végétaux ainsi que la population pour chaque pays. C'est ce DataFrame qui servira à la réalisation des questions ci-dessous.

In [ ]:
# On crée deux dataframes 'bilan_alim' auxquelles on ajoute une colonne 'origine'
bilan_alim_anim = animaux_fao
bilan_alim_anim['origine'] = 'animal'
bilan_alim_veg = vegetaux_fao
bilan_alim_veg['origine'] = 'vegetal'

# On effectue une jointure entre les deux tables et on supprime les tables
bilan_alim = bilan_alim_anim.append(bilan_alim_veg)
del bilan_alim_anim
del bilan_alim_veg

# On modifie la table de manière à ce que chaque élément devienne une colonne
# On renomme les colonnes du nouveau DataFrame
bilan_alim = pd.pivot_table(bilan_alim, index=['Code zone', 'Zone', 'Code Produit', 'Produit', 'origine'], columns = 'Élément', values = 'Valeur', aggfunc=np.sum).reset_index()
bilan_alim.columns.name = None
bilan_alim = bilan_alim.rename(columns = {'Zone' : 'pays', 
                                          'Code zone' : 'code_pays', 
                                          'Produit' : 'produit', 
                                          'Code Produit' : 'code_produit', 
                                          'Année' : 'annee',
                                          'Aliments pour animaux' : 'alim_animaux',
                                          'Autres utilisations (non alimentaire)' : 'autres',
                                          'Disponibilité alimentaire (Kcal/personne/jour)' : 'dispo_kcal_p_j',
                                          'Disponibilité alimentaire en quantité (kg/personne/an)' : 'dispo_kg_p',
                                          'Disponibilité de matière grasse en quantité (g/personne/jour)' : 'dispo_mat_g_p_j',
                                          'Disponibilité de protéines en quantité (g/personne/jour)' : 'dispo_prot_g_p_j',
                                          'Disponibilité intérieure': 'dispo_int',
                                          'Exportations - Quantité' : 'exports',
                                          'Importations - Quantité' : 'imports',
                                          'Nourriture' : 'nourriture',
                                          'Pertes' : 'pertes',
                                          'Production' : 'prod',
                                          'Semences' : 'semences',
                                          'Traitement' : 'traitement',
                                          'Variation de stock' : 'var_stock'})

#On effectue une jointure entre le Dataframe bilan_alim et population
bilan_alim = pd.merge(bilan_alim, population, on =['pays', 'code_pays'])

-------------------------------------------------------------------------------------------------------------------

*Parmi les documents sur les Bilans alimentaires que vous avez téléchargés, il y a des informations redondantes. En effet, pour un pays donné, certaines de ces informations peuvent se calculer à partir d'autres : productions, importations, exportations, variation de stock, disponibilité intérieure, semences, pertes, nourriture (ou disponibilité alimentaire), aliments pour animaux, traitement et autres utilisations.*

**Question 2** : Identifiez ces redondances, en donnant votre réponse sous forme de formule mathématique. C'est une équation à 3 termes de type : (a_1 + a2 + [...] = b_1 + b_2 + [...] = c_1 + c_2 + [...]) faisant intervenir chacune des 11 quantités données ci dessus. Illustrez cette équation avec l'exemple du blé en France.

Selon la méthodologie utilisée par la FAO lors de la récolte des données jusqu'en 2013, il faut distinguer la consommation intérieure (aussi appelée utilisations) et la disponibilité intérieure (aussi appelée nourriture). A partir de ces informations, on peut en déduire que :

- Consommation intérieure = aliment pour animaux + nourriture + semences + pertes + traitement + autres utilisations

- Disponibilité intérieure = Production + (importations - exportations) + variation de stocks = Consommation intérieure

On utilise l'exemple du blé en France pour illustrer cette équation.

In [ ]:
# On sélectionne les données du blé en France pour créer un Dataframe
ble_france = bilan_alim.loc[(bilan_alim['pays'] == 'France') & (bilan_alim['produit'] == 'Blé')]

# On calcule la disponibilité intérieure du blé en France
dispo_int_fb = ble_france['dispo_int'].sum()
print('Disponibilité intérieure du blé en France : ' + str(dispo_int_fb))

# On calcule la consommation intérieure à partir de l'alimentation aniamle, la nourriture, les semences,  les pertes, le traitement et autres utilisations
conso_int_fb = ble_france['alim_animaux'].sum() + ble_france['nourriture'].sum() + ble_france['semences'].sum() + ble_france['pertes'].sum() + ble_france['traitement'].sum() + ble_france['autres'].sum()
print('Consommation intérieure de blé en France : ' + str(conso_int_fb))

# On calcule la disponibilité intérieure à partir de la production, des imports, des exports de de la variation de stocks
prod_fb = ble_france['prod'].sum() + (ble_france['imports'].sum() - ble_france['exports'].sum()) + ble_france['var_stock'].sum()
print('Production + imports/exports + variation de stock de blé en France : ' + str(prod_fb))

Disponibilité intérieure du blé en France : 20298.0
Consommation intérieure de blé en France : 20298.0
Production + imports/exports + variation de stock de blé en France : 20298.0


**Question 3** : Calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines.Vous ferez cela à partir de ces informations :
- Population de chaque pays ;
- Disponibilité alimentaire donnée pour chaque produit et pour chaque pays en kcal/personne/jour.
- Disponibilité alimentaire en protéines donnée pour chaque produit et pour chaque pays en g/personne/jour

In [ ]:
# On crée trois constantes pour pouvoir convertir les unités dans la suite
NB_DE_JOURS = 365
CONVERSION_MILLIERS_TONNES_EN_KG = 1000000
CONVERSION_G_EN_KG = 1000

In [ ]:
# On crée une colonne contenant la disponibilité alimentaire en kcal total pour chaque pays et chaque produit
bilan_alim['dispo_alim_kcal'] = bilan_alim['dispo_kcal_p_j']* (bilan_alim['population']*1000)* 365

In [ ]:
# On crée une colonne contenant la disponibilité alimentaire en protéines pour chaque pays et chaque produit
bilan_alim['dispo_alim_prot'] = (bilan_alim['dispo_prot_g_p_j'] / CONVERSION_G_EN_KG) * (bilan_alim['population']*1000) * 365

**Question 4** : A partir de ces dernières informations, et à partir du poids de la disponibilité alimentaire (pour chaque pays et chaque produit), calculez pour chaque produit le ratio "énergie/poids", que vous donnerez en kcal/kg. Vous pouvez vérifier la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet, par exemple en cherchant la valeur calorique d'un oeuf.

In [ ]:
# On ajoute une colonne contenant le ratio énergie/poids
bilan_alim['ratio_energie'] = bilan_alim['dispo_alim_kcal']/(bilan_alim['dispo_kg_p']*1000*bilan_alim['population'])

*En suivant la même méthodologie, calculez également le pourcentage de protéines de chaque produit (pour chaque pays). Ce pourcentage est obtenu en calculant le ratio "poids de protéines/poids total" (attention aux unités utilisées). Vous pouvez vérifier la cohérence de votre calcul en comparant ce ratio aux données disponibles sur internet, par exemple en cherchant la teneur en protéines de l'avoine.*

In [ ]:
# On ajoute une colonne contenant le pourcentage de protéines
bilan_alim['proteines (%)'] = (bilan_alim['dispo_alim_prot'] /(bilan_alim['dispo_kg_p']*1000*bilan_alim['population'])) * 100

**Question 5** : Citez 5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids.

In [ ]:
# On regroupe les aliments les plus caloriques pour avoir une valeur moyenne pour chaque produit
# On remplace les valeurs infinis par des NaN
# On affiche les 20 premiers produits les plus caloriques
dispo_moy_energie = bilan_alim.groupby(bilan_alim['produit']).agg('mean')
dispo_moy_energie = dispo_moy_energie.replace([-np.inf, np.inf], np.nan)
dispo_moy_energie = dispo_moy_energie.sort_values('ratio_energie', ascending = False)
dispo_moy_energie = dispo_moy_energie[['ratio_energie', 'proteines (%)']].reset_index()
print(dispo_moy_energie.iloc[0:20])

produit  ratio_energie  proteines (%)
0            Huile de Palme    8791.217534       0.391267
1             Huile de Soja    8625.483876       0.201601
2       Huile de Son de Riz    8497.969939       0.688679
3   Huile de Colza&Moutarde    8368.869607       0.239452
4        Huile de Tournesol    8305.568299       0.131633
5   Huil Plantes Oleif Autr    8246.215771       0.571416
6    Huile Graines de Coton    8177.340910       0.178576
7             Huile de Coco    8097.752477       0.435161
8        Huile de Palmistes    7885.602997       0.268712
9    Huile de Germe de Maïs    7827.167158       0.190793
10         Huile d'Arachide    7803.918649       0.126736
11            Huile d'Olive    7774.263485       0.000000
12   Graisses Animales Crue    7579.278844       1.352675
13          Huile de Sésame    7482.722820       0.000000
14       Huiles de Poissons    7133.945055       0.000000
15             Beurre, Ghee    6946.278887       0.633141
16   Arachides Decortiquees    546

*Citez 5 aliments parmi les 20 aliments les plus riches en protéines.*

In [ ]:
# Même méthodologie que pour les calories appliquées aux protéines
dispo_moy_prot = bilan_alim.groupby(['produit']).agg('mean')
dispo_moy_prot = dispo_moy_prot.replace([-np.inf, np.inf], np.nan)
dispo_moy_prot = dispo_moy_prot.sort_values('proteines (%)', ascending = False)
dispo_moy_prot = dispo_moy_prot[['ratio_energie', 'proteines (%)']].reset_index()
print(dispo_moy_prot.iloc[0:20])

produit  ratio_energie  proteines (%)
0                       Soja    1748.819646      25.338211
1     Arachides Decortiquees    5466.324644      24.790816
2                       Pois    3106.951940      22.108112
3        Légumineuses Autres    3170.465864      22.025876
4                   Haricots    3256.453914      21.719055
5                     Sésame    5010.932143      17.828134
6           Abats Comestible    1125.965304      17.630901
7              Viande, Autre    1086.200200      17.410158
8   Plantes Oleiferes, Autre    2631.827466      16.693495
9      Aliments pour enfants    3050.469022      14.872622
10          Viande de Bovins    1689.213495      14.461108
11              Cephalopodes     376.592794      14.057333
12    Viande d'Ovins/Caprins    1933.424399      13.736265
13       Viande de Volailles    1301.168956      12.620016
14       Poissons Pelagiques     888.304389      12.129966
15                   Piments    2514.499721      11.337096
16    Graines Colz

**Question 6** : Calculez, pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée en kcal.

In [ ]:
# On crée un dataframe 'bilan_alim_veg' en ne conservant que les données d'origine végétale
bilan_alim_veg = bilan_alim.loc[bilan_alim['origine'] == 'vegetal']
bilan_alim_veg = bilan_alim_veg.replace([-np.inf, np.inf], np.nan)

# On calcule le nombre de calories totales en utilisant la disponibilité intérieure mondiale
bilan_alim_veg['dispo_int_kcal'] = (bilan_alim_veg['dispo_int'] * CONVERSION_MILLIERS_TONNES_EN_KG) * bilan_alim_veg['ratio_energie']
dispo_int_veg_kcal = round(bilan_alim_veg['dispo_int_kcal'].sum(skipna=True),2)
print('Disponibilité intérieure issue des produits d\'origine végétale : ' + str(dispo_int_veg_kcal) + ' kcal.')

Disponibilité intérieure issue des produits d'origine végétale : 1.2166421524173388e+16 kcal.


**Question 7** : Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [ ]:
# On considère que la consommation moyenne en calories d'un être humain est de 2000 kcal
nombre_humains_kcal = bilan_alim_veg['dispo_int_kcal'].sum(skipna=True) / (2000 * NB_DE_JOURS)
pourcentage_humains_kcal = (nombre_humains_kcal /(population['population'].sum(skipna=True)*1000))* 100
print('Pourcentage d\'humains nourris avec les calories de la disponibilité intérieure: ' 
     + str(pourcentage_humains_kcal) + " %")

# On effectue le même calcul pour les protéines
bilan_alim_veg['dispo_int_prot'] = (bilan_alim_veg['dispo_int'] * CONVERSION_MILLIERS_TONNES_EN_KG) * (bilan_alim_veg['proteines (%)']/100)
dispo_int_veg_prot = round(bilan_alim_veg['dispo_int_prot'].sum(skipna=True),2)
print('Disponibilité intérieure issue des produits d\'origine végétale : ' + str(dispo_int_veg_prot) + ' kg de protéines.')

# On considère que la consommation moyenne en protéines d'un être humain est de 50 g par jour
nbre_humains_proteine = dispo_int_veg_prot / (50 * CONVERSION_G_EN_KG * NB_DE_JOURS)
pourcentage_humains_proteine = (nbre_humains_proteine / population['population'].sum(skipna=True)*1000) * 100
print('Pourcentage d\'humains nourris  avec les protéines de la disponibilité intérieure: '  
     + str(pourcentage_humains_proteine) +  ' %')

Pourcentage d'humains nourris avec les calories de la disponibilité intérieure: 238.1814260909387 %
Disponibilité intérieure issue des produits d'origine végétale : 291995825800.39 kg de protéines.
Pourcentage d'humains nourris  avec les protéines de la disponibilité intérieure: 228.65550750006074 %


**Question 8** : Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux la nourriture végétale destinée aux animaux et les pertes de produits végétaux étaient utilisés pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [ ]:
bilan_alim_veg['gasp_kg'] = (bilan_alim_veg['nourriture'] + bilan_alim_veg['pertes'] + bilan_alim_veg['alim_animaux']) * CONVERSION_MILLIERS_TONNES_EN_KG
bilan_alim_veg['gasp_kcal'] = bilan_alim_veg['gasp_kg'] * bilan_alim_veg['ratio_energie']
gasp_kcal_veg = bilan_alim_veg['gasp_kcal'].sum(skipna=True)
print('Disponibilité alimentaire (nourriture, pertes et aliments pour animaux) : ' + str(gasp_kcal_veg) + ' kcal.')
nb_humains_kcal_veg = gasp_kcal_veg / (2000 * NB_DE_JOURS)
pourcentage_humains_kcal_veg =  (nb_humains_kcal_veg / (population['population'].sum(skipna=True)*1000)) * 100
print('Pourcentage d\'humains nourris avec la nourriture, les pertes et l\'alimentation animale : ' 
     + str(pourcentage_humains_kcal_veg) + " %")

bilan_alim_veg['gasp_prot_kcal'] = bilan_alim_veg['gasp_kg'] * (bilan_alim_veg['proteines (%)'] / 100)
gasp_prot_veg = bilan_alim_veg['gasp_prot_kcal'].sum(skipna=True)
print('Disponibilité alimentaire (nourriture, pertes et aliments pour animaux) : ' + str(gasp_prot_veg) + ' kg de protéines.')
nb_humains_prot_veg = bilan_alim_veg['gasp_prot_kcal'].sum(skipna=True) / ((50 / CONVERSION_G_EN_KG) * NB_DE_JOURS)
pourcentage_humains_proteine_veg = (nb_humains_prot_veg / (population['population'].sum(skipna=True)*1000))*100
print('Pourcentage d\'humains nourris avec la nourriture, les pertes et l\'alimentation animale : ' 
     + str(pourcentage_humains_proteine_veg) + " %")

Disponibilité alimentaire (nourriture, pertes et aliments pour animaux) : 6103854708181490.0 kcal.
Pourcentage d'humains nourris avec la nourriture, les pertes et l'alimentation animale : 119.49485854636568 %
Disponibilité alimentaire (nourriture, pertes et aliments pour animaux) : 161047080637.75153 kg de protéines.
Pourcentage d'humains nourris avec la nourriture, les pertes et l'alimentation animale : 126.11242593516243 %


**Question 9** : Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [ ]:
bilan_alim['nourriture_kcal'] = (bilan_alim['nourriture']*CONVERSION_MILLIERS_TONNES_EN_KG) * bilan_alim['ratio_energie']
bilan_alim = bilan_alim.replace([-np.inf, np.inf], np.nan)
nourriture_tot = bilan_alim['nourriture_kcal'].sum(skipna=True)
print('Nourriture d\'origine animale et végétale : ' + str(nourriture_tot) + ' kcal.')
nb_humain_kcal_nourriture = bilan_alim['nourriture_kcal'].sum(skipna=True) / (2000 * NB_DE_JOURS)
pourcentage_humains_kcal_nourriture =  (nb_humain_kcal_nourriture / (population['population'].sum(skipna=True)*1000)) * 100
print('Pourcentage d\'humains nourris avec la nourriture d\'origine animale et végétale : ' 
     + str(pourcentage_humains_kcal_nourriture) + " %")


bilan_alim['nourriture_prot'] = bilan_alim['nourriture'] * (bilan_alim['proteines (%)']/100)
nourriture_prot_tot = bilan_alim['nourriture_prot'].sum(skipna=True)
print('Nourriture d\'origine animale et végétale : ' + str(nourriture_prot_tot) + ' kg de protéines.')
nb_humains_proteine_nourriture = bilan_alim['nourriture_prot'].sum(skipna=True) / ((50 / CONVERSION_G_EN_KG)* NB_DE_JOURS)
pourcentage_humains_proteine_nourriture = (nb_humains_proteine_nourriture/ population['population'].sum(skipna=True)*1000)  * 100
print('Pourcentage d\'humains nourris avec la nourriture d\'origine animale et végétale : ' 
     + str(pourcentage_humains_proteine_nourriture) + " %")

Nourriture d'origine animale et végétale : 7357936865086520.0 kcal.
Pourcentage d'humains nourris avec la nourriture d'origine animale et végétale : 144.04596225203272 %
Nourriture d'origine animale et végétale : 207153.4766853036 kg de protéines.
Pourcentage d'humains nourris avec la nourriture d'origine animale et végétale : 162.21733037464818 %


**Question 10** : A partir des données téléchargées qui concernent la sous-nutrition, répondez à cette question : Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

In [ ]:
sous_nutrition = sousalim_fao
sous_nutrition = sous_nutrition.loc[sous_nutrition['Année'] == '2012-2014']
sous_nutrition = sous_nutrition.replace('<0.1', 0.1)
sous_nutrition = sous_nutrition.drop(columns= ['Code Domaine', 'Domaine', 'Code Élément', 'Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité', 'Symbole', 'Description du Symbole', 'Note'])
sous_nutrition = sous_nutrition.rename(columns = {'Code zone' : 'code_pays',
                                                  'Zone' : 'pays',
                                                  'Valeur' : 'nb_habitants'})
sous_nutrition = pd.merge(sous_nutrition, population, on=['pays', 'code_pays'])
sous_nutrition = sous_nutrition.astype({'nb_habitants' : 'float'})

pourcent_ss_nutrition = round(((sous_nutrition['nb_habitants'].sum(skipna=True)*1000)/(sous_nutrition['population'].sum(skipna=True))*100),2)
print('Nombre de personnes sous-alimentés : ' + str(sous_nutrition['nb_habitants'].sum(skipna=True)) + ' milliers.')
print('Pourcentages de personnes sous-alimentés : ' + str(pourcent_ss_nutrition) + '%')

Nombre de personnes sous-alimentés : 746.0 milliers.
Pourcentages de personnes sous-alimentés : 10.66%


*Établissez la liste des produits (ainsi que leur code) considéré comme des céréales selon la FAO.*

In [ ]:
cereales_consommable = cereales_fao[['Produit','Code Produit']]
cereales_consommable = cereales_consommable.drop_duplicates()
cereales_consommable = cereales_consommable.rename(columns = {'Produit' : 'produit',
                                                              'Code Produit' : 'code_produit'})
print(cereales_consommable)

             produit  code_produit
0                Blé          2511
1   Riz (Eq Blanchi)          2805
2               Orge          2513
3               Maïs          2514
4             Millet          2517
9             Seigle          2515
10            Avoine          2516
12            Sorgho          2518
13  Céréales, Autres          2520


**Question 11** : En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), quelle proportion (en termes de poids) est destinée à l'alimentation animale ?

In [ ]:
cereales_alim = pd.merge(cereales_consommable,  bilan_alim_veg, on = 'produit')
cereales_tot_alim = cereales_alim['nourriture'].sum(skipna=True)
cereales_tot_anim = cereales_alim['alim_animaux'].sum(skipna=True)
print("Quantité de céréales nécessaires à l'alimentation totale : " + str(cereales_tot_alim) + " milliers de tonnes.\nQuantité de céréales nécessaires à l'alimentation animale : " + str(cereales_tot_anim) + " milliers de tonnes.")
print("Quantité de céréales restantes une fois l'alimentation animale enlevée : " + str(abs(cereales_tot_alim - cereales_tot_anim)) + " milliers de tonnes.")
ratio = cereales_tot_anim/(cereales_tot_alim + cereales_tot_anim) * 100
ratio = round(ratio, 2)
print("Ratio alimentation animale/alimentation totale : " + str(ratio) + "%")

Quantité de céréales nécessaires à l'alimentation totale : 1029010.0 milliers de tonnes.
Quantité de céréales nécessaires à l'alimentation animale : 873535.0 milliers de tonnes.
Quantité de céréales restantes une fois l'alimentation animale enlevée : 155475.0 milliers de tonnes.
Ratio alimentation animale/alimentation totale : 45.91%


*Sélectionnez parmi les données des bilans alimentaires les informations relatives aux pays dans lesquels la FAO recense des personnes en sous-nutrition. Repérez les 15 produits les plus exportés par ce groupe de pays.*

In [ ]:
sous_nutrition = sous_nutrition.sort_values('nb_habitants', ascending = False)
sous_nutrition = sous_nutrition.dropna()
bilan_alim_ss_nutrition = pd.merge(bilan_alim, sous_nutrition, on=['code_pays', 'pays', 'annee', 'population'])
bilan_alim_ss_nutrition = bilan_alim_ss_nutrition.groupby(['produit']).agg('sum')
bilan_alim_ss_nutrition = bilan_alim_ss_nutrition.sort_values('exports', ascending=False)
bilan_alim_ss_nutrition = bilan_alim_ss_nutrition.head(15)
bilan_alim_ss_nutrition = bilan_alim_ss_nutrition.reset_index()
print(bilan_alim_ss_nutrition['produit'])
produit_export_ss_alim = pd.DataFrame(bilan_alim_ss_nutrition['produit'])

0          Huile de Palme
1                    Maïs
2                  Manioc
3        Riz (Eq Blanchi)
4           Sucre Eq Brut
5                     Blé
6         Légumes, Autres
7                 Bananes
8      Lait - Excl Beurre
9          Fruits, Autres
10                   Soja
11    Poissons Pelagiques
12                Tomates
13                 Pommes
14    Oranges, Mandarines
Name: produit, dtype: object


*Parmi les données des bilans alimentaires au niveau mondial, sélectionnez les 200 plus grandes importations de ces produits (1 importation = une quantité d'un produit donné importée par un pays donné).*

In [ ]:
bilan_alim_imports = pd.merge(bilan_alim, produit_export_ss_alim, on = ['produit'])
bilan_alim_imports = bilan_alim_imports.sort_values('imports', ascending=False)
bilan_alim_imports = bilan_alim_imports.iloc[0:200]
print(bilan_alim_imports[['code_pays', 'pays', 'produit', 'imports']])

      code_pays                 pays             produit  imports
714          41  Chine, continentale                Soja  63381.0
375          41  Chine, continentale              Manioc  29046.0
251         110                Japon                Maïs  14403.0
41           59               Égypte                 Blé  10331.0
257         117  République de Corée                Maïs   8755.0
...         ...                  ...                 ...      ...
2328        131             Malaisie    Riz (Eq Blanchi)   1079.0
1271        106               Italie     Légumes, Autres   1061.0
2572        237             Viet Nam  Lait - Excl Beurre   1041.0
122         173              Pologne                 Blé   1037.0
196          33               Canada                Maïs   1035.0

[200 rows x 4 columns]


*Groupez ces importations par produit, afin d'avoir une table contenant 1 ligne pour chacun des 15 produits.*

In [ ]:
bilan_alim_imports = bilan_alim_imports.groupby(['produit']).mean()
bilan_alim_imports = bilan_alim_imports.reset_index()
print(bilan_alim_imports[['produit', 'imports']])

                produit      imports
0               Bananes  1932.600000
1                   Blé  3432.000000
2        Fruits, Autres  2244.857143
3        Huile de Palme  2974.700000
4    Lait - Excl Beurre  3274.740741
5       Légumes, Autres  2561.545455
6                Manioc  6931.000000
7                  Maïs  3658.964286
8   Oranges, Mandarines  2868.125000
9   Poissons Pelagiques  2240.833333
10               Pommes  1703.600000
11     Riz (Eq Blanchi)  1554.444444
12                 Soja  6230.333333
13        Sucre Eq Brut  2138.000000
14              Tomates  1428.600000


*Ensuite, calculez pour chaque produit les 2 quantités suivantes :*
- le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.
- le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)

In [ ]:
bilan_alim_imports['ratio(autres/dispo_int)'] = bilan_alim_imports['autres']/bilan_alim_imports['dispo_int']
bilan_alim_imports['ratio(alim_anim/nourriture)'] = bilan_alim_imports['alim_animaux']/bilan_alim_imports['nourriture']
bilan_alim_imports = bilan_alim_imports.replace([-np.inf, np.inf], np.nan)

**Question 12** : Donnez les 3 produits qui ont la plus grande valeur pour chacun des 2 ratios (vous aurez donc 6 produits à citer)

In [ ]:
bilan_alim_imports = bilan_alim_imports.sort_values('ratio(autres/dispo_int)', ascending = False)
print(bilan_alim_imports['produit'].iloc[0:3])

3    Huile de Palme
7              Maïs
6            Manioc
Name: produit, dtype: object


In [ ]:
bilan_alim_imports = bilan_alim_imports.sort_values('ratio(alim_anim/nourriture)', ascending = False)
print(bilan_alim_imports['produit'].iloc[0:3])

7                    Maïs
9     Poissons Pelagiques
12                   Soja
Name: produit, dtype: object


**Question 13** : Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [ ]:
cereales_usa = bilan_alim.loc[bilan_alim['pays'] == 'États-Unis d\'Amérique']
cereales_usa = pd.merge(cereales_consommable, cereales_usa, on = ['produit', 'code_produit'])
cereales_tot_usa = cereales_usa['alim_animaux'].sum(skipna=True)
print('Production totale de céréales pour l\'alimentation animale : ' + str(cereales_tot_usa) + ' milliers de tonnes par an.')

pourcent_tot_usa = cereales_tot_usa * 0.01
print('Libération de 10% de la production :  ' + str(pourcent_tot_usa) + ' milliers de tonnes de céréales par an.')

Production totale de céréales pour l'alimentation animale : 140096.0 milliers de tonnes par an.
Libération de 10% de la production :  1400.96 milliers de tonnes de céréales par an.


In [ ]:
cereales_usa['alim_animaux_kcal'] = (cereales_usa['alim_animaux']*CONVERSION_MILLIERS_TONNES_EN_KG) * bilan_alim['ratio_energie']
cereales_kcal = (cereales_usa['alim_animaux_kcal'].sum(skipna=True) * 0.01)
nb_humain_kcal_cereales = round(cereales_kcal / (2000 * NB_DE_JOURS),2)
pourcentage_humains_kcal_cereales =  round((nb_humain_kcal_cereales / (population['population'].sum(skipna=True)*1000)) * 100,2)
print('En libérant 10% de la production céréalière des USA destinée à l\'alimentation animale, on pourrait nourrir ' +str(nb_humain_kcal_cereales) + ' personnes soit '+ str(pourcentage_humains_kcal_cereales) + '% de la population mondiale.')

En libérant 10% de la production céréalière des USA destinée à l'alimentation animale, on pourrait nourrir 5672552.24 personnes soit 0.08% de la population mondiale.


**Question 14** : En Thaïlande, quelle proportion de manioc est exportée ? Quelle est la proportion de personnes en sous-nutrition?

In [ ]:
thailande = bilan_alim.loc[(bilan_alim['pays'] == 'Thaïlande') & (bilan_alim['produit'] == 'Manioc')]
export_manioc_thailande = thailande['exports'].sum(skipna=True)
print('Proportion de manioc exportée par la Thaïlande : ' + str(export_manioc_thailande) + ' milliers de tonnes.')
thailande = pd.merge(thailande, sous_nutrition, on = ['pays', 'code_pays', 'annee', 'population'])
pourcent_ss_nutr_thailande = round(((thailande['nb_habitants'].sum(skipna=True)* 1000) / thailande['population'].sum(skipna=True)) * 100, 2)
print('Pourcentage de thaïlandais sous-alimentés : ' + str(pourcent_ss_nutr_thailande) +'%')

Proportion de manioc exportée par la Thaïlande : 25214.0 milliers de tonnes.
Pourcentage de thaïlandais sous-alimentés : 8.36%


In [ ]:
thailande['kcal_exports'] = (thailande['exports'] * CONVERSION_MILLIERS_TONNES_EN_KG) * bilan_alim['ratio_energie']
manioc_kcal = thailande['kcal_exports'].sum(skipna=True)
nb_humain_manioc = round(manioc_kcal/(2000 * NB_DE_JOURS),2)
pourcentage_hab_manioc = round((nb_humain_manioc/67011000) * 100,2)
print('Cette quantité de manioc pourrait nourrir ' + str(nb_humain_manioc) + ' thaïlandais soit ' + str(pourcentage_hab_manioc) + ' % de la population thaïlandaise.')

Cette quantité de manioc pourrait nourrir 98848147.01 thaïlandais soit 147.51 % de la population thaïlandaise.
